In [1]:
import polars as pl
import sys
import os
import datetime

sys.path.append(os.path.dirname(os.path.abspath('')))

from libraries.client_stashapp import get_stashapp_client, StashAppClient

stash = get_stashapp_client()
stash_client = StashAppClient()

dUsing stash (v0.28.1-89-g642b0f22) endpoint at https://stash.chiefsclub.com:443/graphql
dUsing stash (v0.28.1-89-g642b0f22) endpoint at https://stash.chiefsclub.com:443/graphql


In [7]:
studio = stash.find_studio("Braless Forever")
studio

dmatched "Braless\ Forever" to "Braless Forever" (1063) using primary name


{'id': '1063',
 'name': 'Braless Forever',
 'url': 'https://bralessforever.com/',
 'parent_studio': None,
 'child_studios': [],
 'aliases': [],
 'tags': [],
 'ignore_auto_tag': False,
 'image_path': 'https://stash.chiefsclub.com/studio/1063/image?t=1718370858',
 'scene_count': 312,
 'image_count': 0,
 'gallery_count': 0,
 'performer_count': 38,
 'group_count': 0,
 'stash_ids': [{'endpoint': 'https://stashdb.org/graphql',
   'stash_id': 'b812347f-3f7c-4a48-a60c-1217d7cc4755',
   'updated_at': '1970-01-01T00:00:00Z'}],
 'rating100': None,
 'favorite': False,
 'details': '',
 'created_at': '2024-06-14T16:14:18+03:00',
 'updated_at': '2024-06-14T16:14:18+03:00',
 'groups': []}

In [25]:
# Read JSON file
import json
import base64

base_dir_path = "/Volumes/Culture 5/Videos/Sites/Braless Forever/"
category = "xxx"
scene_id = "0dba2428-78f3-434d-a1c6-4dbe212a30dd"
dir_path = os.path.join(base_dir_path, category, scene_id)
json_file_path = os.path.join(dir_path, f"{scene_id}.json")

with open(json_file_path, 'r') as f:
    data = json.load(f)

# Read cover image
image_path = os.path.join(dir_path, f"{scene_id}.jpg")
with open(image_path, 'rb') as img_file:
    base64_image = base64.b64encode(img_file.read()).decode('utf-8')
    cover_image_base64 = f"data:image/jpeg;base64,{base64_image}"

print(f"Loaded JSON data for scene {scene_id}")
print(f"Loaded cover image from {image_path}")
print(data)

Loaded JSON data for scene 0dba2428-78f3-434d-a1c6-4dbe212a30dd
Loaded cover image from /Volumes/Culture 5/Videos/Sites/Braless Forever/xxx/0dba2428-78f3-434d-a1c6-4dbe212a30dd/0dba2428-78f3-434d-a1c6-4dbe212a30dd.jpg
{'@context': 'https://schema.org', '@type': 'VideoObject', 'name': 'Getting Handsy at the Beach', 'description': 'Rocky jerks off her husband with a stranger watching and she shows off for him to make him cum! ', 'image': 'https://public-blvideo.b-cdn.net/RockyXXXBeachjerk/thumbnail-dc14e02e9e.jpg', 'thumbnailUrl': 'https://public-blvideo.b-cdn.net/RockyXXXBeachjerk/thumbnail-dc14e02e9e.jpg', 'contentUrl': 'https://app.bralessforever.com/videos/0dba2428-78f3-434d-a1c6-4dbe212a30dd', 'embedUrl': 'https://app.bralessforever.com/embed/video/0dba2428-78f3-434d-a1c6-4dbe212a30dd', 'uploadDate': '2021-11-05T16:06:27.000Z', 'duration': 'PT1097S', 'actor': [{'@type': 'Person', 'name': 'Rocky'}], 'interactionStatistic': {'@type': 'InteractionCounter', 'interactionType': {'@type': 

In [26]:
scenes = stash.find_scenes({ "path": { "value": scene_id, "modifier": "INCLUDES" } }, fragment="id title")
scenes

[{'id': '34318', 'title': ''}]

In [27]:
scene_actors = []

if "actor" in data and data["actor"]:
    for actor in data["actor"]:
        braless_name = "Braless Forever: " + actor["name"]
        stash_actor = stash.find_performer(braless_name)
        if not stash_actor:
            stash_actor = stash.create_performer({ "name": braless_name })
        scene_actors.append(stash_actor["id"])

In [28]:
stash.update_scene({
    "id": scenes[0]["id"],
    "title": data["name"],
    "details": data["description"],
    "urls": [data["contentUrl"]],
    "date": datetime.datetime.fromisoformat(data["uploadDate"].replace('Z', '+00:00')).date().isoformat(),
    "performer_ids": scene_actors,
    "cover_image": cover_image_base64,
    "studio_id": studio["id"]
})

'34318'